# Exercise 1

In [1]:
# import libraries and read xlsx into df's

import pandas as pd
import numpy as np
import timeit

xlsx = pd.ExcelFile('data.xlsx')
forecast = pd.read_excel(xlsx, sheet_name=0)
meters = pd.read_excel(xlsx, sheet_name=1)
rates = pd.read_excel(xlsx, sheet_name=2)

# enrich forecast dataframe with exit zone in order to have a key to join on
forecast_enriched = pd.merge(forecast, meters, on='Meter ID')

# join rates on meters and drop records for which annual quantity (kwh) is 
# out of range in order to have the appropriate bands

rates = pd.merge(rates, meters[['Exit Zone','Annual Quantity (kWh)']], on='Exit Zone')
rates = rates[rates['Annual Quantity (kWh)'].between(rates['Annual Quantity (Min)'],rates['Annual Quantity (Max)'])]

# join forecast on rates by exit zone on date with backwards direction

forecast_rated = pd.merge_asof(
                    forecast_enriched.sort_values('Date'),
                    rates.sort_values('Date'),
                    by='Exit Zone', 
                    on='Date', 
                    direction='backward'
                    )

# calculate cost per day in £

forecast_rated['cost_per_day'] = forecast_rated['kWh'].mul(forecast_rated['Rate (p/kWh)']).div(100)

# create dataframe containing total estimated consumption and total cost (£)

total_cost = forecast_rated.groupby('Meter ID').agg(
    total_estimated_consumption=pd.NamedAgg(column="kWh", aggfunc="sum"),
    total_cost=pd.NamedAgg(column="cost_per_day", aggfunc="sum"))
        
# round up to 2 decimals and reformat 

total_cost = total_cost.round(2).reset_index()
total_cost
        


,Meter ID,total_estimated_consumption,total_cost
0,10588707,122450.0,28.50
1,10626610,303558.0,88.24
2,1000000603,24674.0,8.27


# Exercise 2 - generate dummy meters

In [3]:
# define function to generate meters

def generate_meters(n):
    meters_generated = pd.DataFrame(dict(
        meter_id=np.random.randint(999999,1111111, size = n),
        annual_quantity_kwh=np.random.randint(10000, 999000, size=n),
        exit_zone= pd.util.testing.rands_array(6, n)
        ))
    return meters_generated
        
# call function to generate meters and store the dataframe for re-use
    
meters_generated = generate_meters(10)
meters_generated


,meter_id,annual_quantity_kwh,exit_zone
0,1033127,859108,KZDfoF
1,1110626,224411,27AUmJ
2,1108660,834456,sfa8TF
3,1080336,952834,3EP1fp
4,1092859,145502,Er1Dsv
5,1087198,750713,SItGSn
6,1049826,54261,oCY2Yc
7,1007341,297975,GVmdis
8,1096839,539541,PKpnQH
9,1062911,848154,kfc3o4


# Exercise 3a - generate dummy forecasts

In [4]:
# define function to create a forecasted consumption

def create_forecast(start_date, end_date):
    generated_forecast = pd.DataFrame([x for x in meters_generated.meter_id], columns=['meter_id'])
    generated_forecast["date"] = generated_forecast.apply(
                                        lambda x: pd.date_range(start_date, end_date), axis=1)
    
    generated_forecast = (generated_forecast.explode("date", ignore_index=True))
    
    generated_forecast["kwh"] = np.random.randint(0,500, size=len(generated_forecast))
    return generated_forecast

# call function to create a forecasted consumption and store the dataframe for re-use

forecasted_consumption_generated = create_forecast("01-01-2020","01-01-2030")
forecasted_consumption_generated


,meter_id,date,kwh
0,1033127,2020-01-01,416
1,1033127,2020-01-02,337
2,1033127,2020-01-03,18
3,1033127,2020-01-04,371
4,1033127,2020-01-05,253
...,...,...,...
36535,1062911,2029-12-28,173
36536,1062911,2029-12-29,404
36537,1062911,2029-12-30,191
36538,1062911,2029-12-31,115


# Exercise 3b - generate dummy rates

In [5]:
# define function to create rates. Create 'tmp' series to be able to easily join with quantity min and max

def create_rates(start_date, end_date):
    generated_rates = pd.DataFrame([x for x in meters_generated.exit_zone], columns=['exit_zone'])
    generated_rates["date"] = generated_rates.apply(
                                        lambda x: pd.date_range(start_date, end_date, freq='6MS'), axis=1
                                          )
    generated_rates = (generated_rates.explode("date", ignore_index=True))
    generated_rates["tmp"] = 1
    generated_rates = pd.merge(generated_rates, 
                               pd.DataFrame({'tmp': 1, 'annual_quantity_min': [0, 73200, 732000], 'annual_quantity_max': [73200, 732000, 999000]}), 
                               on='tmp')
    generated_rates.drop(columns=['tmp'])
    generated_rates["rate"] = np.random.random(size=len(generated_rates))
    return generated_rates

# call function to create rates and store the dataframe for re-use

rates_generated = create_rates("04-01-2020","04-01-2030")
rates_generated


,exit_zone,date,tmp,annual_quantity_min,annual_quantity_max,rate
0,KZDfoF,2020-04-01,1,0,73200,0.467971
1,KZDfoF,2020-04-01,1,73200,732000,0.418159
2,KZDfoF,2020-04-01,1,732000,999000,0.520038
3,KZDfoF,2020-10-01,1,0,73200,0.015238
4,KZDfoF,2020-10-01,1,73200,732000,0.133980
...,...,...,...,...,...,...
625,kfc3o4,2029-10-01,1,73200,732000,0.938261
626,kfc3o4,2029-10-01,1,732000,999000,0.557749
627,kfc3o4,2030-04-01,1,0,73200,0.477251
628,kfc3o4,2030-04-01,1,73200,732000,0.823090


# Exercise 4a - create function which calculates the total cost dataframe based on the generated meters, forecast and rates from exercice 2 & 3

In [6]:
# define function

def total_cost_calculator(meters, forecast, rates):

    forecast_enriched = pd.merge(forecast, meters, on='meter_id')

    # merge rates with meters and retain rows for which the range is right

    rates = pd.merge(rates, meters[['exit_zone','annual_quantity_kwh']], on='exit_zone')
    rates = rates[rates['annual_quantity_kwh'].between(rates['annual_quantity_min'],rates['annual_quantity_max'])]

    # Merge by exit zone on date with backwards direction

    forecast_rated = pd.merge_asof(
                    forecast_enriched.sort_values('date'),
                    rates.sort_values('date'),
                    by='exit_zone', 
                    on='date', 
                    direction='backward'
                    )

    forecast_rated['cost_per_day'] = forecast_rated['kwh'].mul(forecast_rated['rate']).div(100)

    total_cost = forecast_rated.groupby('meter_id').agg(
    total_estimated_consumption=pd.NamedAgg(column="kwh", aggfunc="sum"),
    total_cost=pd.NamedAgg(column="cost_per_day", aggfunc="sum"))
        
    total_cost = total_cost.round(2).reset_index()
    
    return total_cost

# call function and store the dataframe for re-use

total_cost_generated = total_cost_calculator(meters_generated, forecasted_consumption_generated, rates_generated)
total_cost_generated




,meter_id,total_estimated_consumption,total_cost
0,1007341,910481,5457.05
1,1033127,910555,4186.17
2,1049826,912747,4825.15
3,1062911,920477,4668.72
4,1080336,921460,5318.22
5,1087198,914464,3469.52
6,1092859,905755,4190.82
7,1096839,915618,3946.62
8,1108660,892162,4624.23
9,1110626,918749,4422.58


# Exercise 4b - create and run the function which calculates the total cost dataframe for which the parameters can be tweaked (amount of meters and time frame)

In [7]:
def total_cost_calculator_schaling(meters_schaling,start_date_schaling, end_date_schaling):
    
# exercise 2
    
    def generate_meters(n):
        meters_generated = pd.DataFrame(dict(
            meter_id=np.random.randint(999999,1111111, size = n),
            annual_quantity_kwh=np.random.randint(10000, 999000, size=n),
            exit_zone= pd.util.testing.rands_array(6, n)
            ))
        return meters_generated
            
    meters_generated = generate_meters(meters_schaling)
    #meters_generated.to_excel(f"meters_{meters_schaling}_{end_date_schaling}_df.xlsx")
    
# exercise 3a


    def create_forecast(start_date, end_date):
        generated_forecast = pd.DataFrame([x for x in meters_generated.meter_id], columns=['meter_id'])
        generated_forecast["date"] = generated_forecast.apply(
            lambda x: pd.date_range(start_date, end_date), axis=1
        )
        generated_forecast = (
            generated_forecast.explode("date", ignore_index=True)
        )

        generated_forecast["kwh"] = np.random.randint(0,500, size=len(generated_forecast))
        return generated_forecast


    forecasted_consumption_generated = create_forecast(start_date_schaling,end_date_schaling)
    #forecasted_consumption_generated.to_excel(f"forecast{meters_schaling}_{end_date_schaling}_df.xlsx")


# exercise 3b

    def create_rates(start_date, end_date):
        generated_rates = pd.DataFrame([x for x in meters_generated.exit_zone], columns=['exit_zone'])
        generated_rates["date"] = generated_rates.apply(
            lambda x: pd.date_range(start_date, end_date, freq='6MS'), axis=1
        )
        generated_rates = (
            generated_rates.explode("date", ignore_index=True)
        )
        generated_rates["tmp"] = 1
        generated_rates = pd.merge(generated_rates, 
                                   pd.DataFrame({'tmp': 1, 'annual_quantity_min': [0, 73200, 732000], 'annual_quantity_max': [73200, 732000, 999000]}), 
                                   on='tmp')
        generated_rates.drop(columns=['tmp'])
        generated_rates["rate"] = np.random.random(size=len(generated_rates))
        return generated_rates

    # call function to create rates

    rates_generated = create_rates(start_date_schaling,end_date_schaling)
    #rates_generated.to_excel(f"rates{meters_schaling}_{end_date_schaling}_df.xlsx")

    
# exercise 4A

    def total_cost_calculator(meters, forecast, rates):

        forecast_enriched = pd.merge(forecast, meters, on='meter_id')

        # merge rates with meters and retain rows for which the range is right

        rates = pd.merge(rates, meters[['exit_zone','annual_quantity_kwh']], on='exit_zone')
        rates = rates[rates['annual_quantity_kwh'].between(rates['annual_quantity_min'],rates['annual_quantity_max'])]

        # Merge by exit zone on date with backwards direction

        forecast_rated = pd.merge_asof(
            forecast_enriched.sort_values('date'),
            rates.sort_values('date'),
            by='exit_zone', 
            on='date', 
            direction='backward'
        )

        forecast_rated['cost_per_day'] = forecast_rated['kwh'].mul(forecast_rated['rate']).div(100)

        total_cost = forecast_rated.groupby('meter_id').agg(
            total_estimated_consumption=pd.NamedAgg(column="kwh", aggfunc="sum"),
            total_cost=pd.NamedAgg(column="cost_per_day", aggfunc="sum"))

        total_cost = total_cost.round(2).reset_index()

        return total_cost


    total_cost_df = total_cost_calculator(meters_generated, forecasted_consumption_generated, rates_generated)
    return total_cost_df

# Run the function with following parameters: 10 meters and data as of 04/2020 up until 04/2025

total_cost_calculator_schaling_df = total_cost_calculator_schaling(10,"04-01-2020","04-01-2025")




# Time the function that reads in the parameter variations from multiple excels files (generated via 4b) and processes this data to print a total cost dataframe

In [8]:
def total_cost_calculator(xlsx1,xlsx2,xlsx3):

    forecast = pd.read_excel(xlsx2, sheet_name=0)
    meters = pd.read_excel(xlsx1, sheet_name=0)
    rates = pd.read_excel(xlsx3, sheet_name=0)
    forecast_enriched = pd.merge(forecast, meters, on='meter_id')

    # merge rates with meters and retain rows for which the range is right

    rates = pd.merge(rates, meters[['exit_zone','annual_quantity_kwh']], on='exit_zone')
    rates = rates[rates['annual_quantity_kwh'].between(rates['annual_quantity_min'],rates['annual_quantity_max'])]

    # Merge by exit zone on date with backwards direction

    forecast_rated = pd.merge_asof(
            forecast_enriched.sort_values('date'),
            rates.sort_values('date'),
            by='exit_zone', 
            on='date', 
            direction='backward'
    )

    forecast_rated['cost_per_day'] = forecast_rated['kwh'].mul(forecast_rated['rate']).div(100)

    total_cost = forecast_rated.groupby('meter_id').agg(
            total_estimated_consumption=pd.NamedAgg(column="kwh", aggfunc="sum"),
            total_cost=pd.NamedAgg(column="cost_per_day", aggfunc="sum"))

    total_cost = total_cost.round(2).reset_index()
    return(total_cost)

# Start by calculating the total time (read+process) for the function, based on several meter&date parameters:
    
print(f"Time to read and process data variation with meters = 10 & max timestamp = 2025")    
print(timeit.timeit(stmt="total_cost_calculator('meters_10_04-01-2025_df.xlsx','forecast10_04-01-2025_df.xlsx','rates10_04-01-2025_df.xlsx')", setup='from __main__ import total_cost_calculator', number=1))
print(f"Time to read and process data variation with meters = 20 & max timestamp = 2025")    
print(timeit.timeit(stmt="total_cost_calculator('meters_20_04-01-2025_df.xlsx','forecast20_04-01-2025_df.xlsx','rates20_04-01-2025_df.xlsx')", setup='from __main__ import total_cost_calculator', number=1))
print(f"Time to read and process data variation with...")    
print(timeit.timeit(stmt="total_cost_calculator('meters_50_04-01-2025_df.xlsx','forecast50_04-01-2025_df.xlsx','rates50_04-01-2025_df.xlsx')", setup='from __main__ import total_cost_calculator', number=1))
print(timeit.timeit(stmt="total_cost_calculator('meters_100_04-01-2025_df.xlsx','forecast100_04-01-2025_df.xlsx','rates100_04-01-2025_df.xlsx')", setup='from __main__ import total_cost_calculator', number=1))
print(timeit.timeit(stmt="total_cost_calculator('meters_10_04-01-2025_df.xlsx','forecast10_04-01-2025_df.xlsx','rates10_04-01-2025_df.xlsx')", setup='from __main__ import total_cost_calculator', number=1))
print(timeit.timeit(stmt="total_cost_calculator('meters_10_04-01-2050_df.xlsx','forecast10_04-01-2050_df.xlsx','rates10_04-01-2050_df.xlsx')", setup='from __main__ import total_cost_calculator', number=1))
print(timeit.timeit(stmt="total_cost_calculator('meters_10_04-01-2100_df.xlsx','forecast10_04-01-2100_df.xlsx','rates10_04-01-2100_df.xlsx')", setup='from __main__ import total_cost_calculator', number=1))
print(timeit.timeit(stmt="total_cost_calculator('meters_50_04-01-2050_df.xlsx','forecast50_04-01-2050_df.xlsx','rates50_04-01-2050_df.xlsx')", setup='from __main__ import total_cost_calculator', number=1))
      

Time to read and process data variation with meters = 10 & max timestamp = 2025
1.7473403929999876
Time to read and process data variation with meters = 20 & max timestamp = 2025
3.391602320000004
Time to read and process data variation with...
10.874772632000003
20.693368549
1.584070742999998
9.290299484000002
25.27674369600001
48.31109802


# Time the function that reads in the data variations from multiple excels (generated via 4b) without processing the data

In [9]:
def total_cost_calculator_read(xlsx1,xlsx2,xlsx3):

    forecast = pd.read_excel(xlsx2, sheet_name=0)
    meters = pd.read_excel(xlsx1, sheet_name=0)
    rates = pd.read_excel(xlsx3, sheet_name=0)
    forecast_enriched = pd.merge(forecast, meters, on='meter_id')

# Next calculate the total time for the read function, based on several meter&date parameters:

print(f"Time to read data variation with meters = 10 & max timestamp = 2025")    
print(timeit.timeit(stmt="total_cost_calculator_read('meters_10_04-01-2025_df.xlsx','forecast10_04-01-2025_df.xlsx','rates10_04-01-2025_df.xlsx')", setup='from __main__ import total_cost_calculator_read', number=1))
print(f"Time to read data variation with meters = 20 & max timestamp = 2025")    
print(timeit.timeit(stmt="total_cost_calculator_read('meters_20_04-01-2025_df.xlsx','forecast20_04-01-2025_df.xlsx','rates20_04-01-2025_df.xlsx')", setup='from __main__ import total_cost_calculator_read', number=1))
print(f"Time to read data variation with meters ...")    
print(timeit.timeit(stmt="total_cost_calculator_read('meters_50_04-01-2025_df.xlsx','forecast50_04-01-2025_df.xlsx','rates50_04-01-2025_df.xlsx')", setup='from __main__ import total_cost_calculator_read', number=1))
print(timeit.timeit(stmt="total_cost_calculator_read('meters_100_04-01-2025_df.xlsx','forecast100_04-01-2025_df.xlsx','rates100_04-01-2025_df.xlsx')", setup='from __main__ import total_cost_calculator_read', number=1))
print(timeit.timeit(stmt="total_cost_calculator_read('meters_10_04-01-2025_df.xlsx','forecast10_04-01-2025_df.xlsx','rates10_04-01-2025_df.xlsx')", setup='from __main__ import total_cost_calculator_read', number=1))
print(timeit.timeit(stmt="total_cost_calculator_read('meters_10_04-01-2050_df.xlsx','forecast10_04-01-2050_df.xlsx','rates10_04-01-2050_df.xlsx')", setup='from __main__ import total_cost_calculator_read', number=1))
print(timeit.timeit(stmt="total_cost_calculator_read('meters_10_04-01-2100_df.xlsx','forecast10_04-01-2100_df.xlsx','rates10_04-01-2100_df.xlsx')", setup='from __main__ import total_cost_calculator_read', number=1))
print(timeit.timeit(stmt="total_cost_calculator_read('meters_50_04-01-2050_df.xlsx','forecast50_04-01-2050_df.xlsx','rates50_04-01-2050_df.xlsx')", setup='from __main__ import total_cost_calculator_read', number=1))

      

Time to read data variation with meters = 10 & max timestamp = 2025
1.9747354040000005
Time to read data variation with meters = 20 & max timestamp = 2025
4.393176304999969
Time to read data variation with meters ...
8.302228368000044
15.228728598000032
1.5547554059999698
10.237703400000044
24.353228618999992
46.12662259299998


In [10]:
print(f"The biggest take-away for me is that, most time is lost with reading in (big) seperate excel files:")
print(f"46.13 of 48.31 seconds is spend reading data. Only 2.18 seconds was spend processing the data") 
print(f"for the data variant with 50 meters and 30 years of data (2020-2050)")
print(f"The processing time for the data seems acceptable to me, so I would be spending my time next")
print(f"into researching possibilities to ingest the data in a better way.")
print(f"CSV should be a better alternative but other datatypes should be able to offer better read times.")

print(f"Next to this, when generating dummy data, it could be interesting to use a seed for reproducibility")
print(f"and to make sure that values can not be re-used (replace=False).")
print(f"In generating the dummy data, some apply/lambda functions were used, which have a negative impact on speed.")

print(f"In order to increase the processing speed of the function, I would look into alternatives for")
print(f"the .between() and the groupby - aggregation. Next to this I would always confirm/coerce that the dates")
      
print(f"are in timestamp format (to_datetime) and ofcourse, always try to avoid (hidden) loops.")


The biggest take-away for me is that, most time is lost with reading in (big) seperate excel files:
46.13 of 48.31 seconds is spend reading data. Only 2.18 seconds was spend processing the data
for the data variant with 50 meters and 30 years of data (2020-2050)
The processing time for the data seems acceptable to me, so I would be spending my time next
into researching possibilities to ingest the data in a better way.
CSV should be a better alternative but other datatypes should be able to offer better read times.
Next to this, when generating dummy data, it could be interesting to use a seed for reproducibility
and to make sure that values can not be re-used (replace=False).
In generating the dummy data, some apply/lambda functions were used, which have a negative impact on speed.
In order to increase the processing speed of the function, I would look into alternatives for
the .between() and the groupby - aggregation. Next to this I would always confirm/coerce that the dates
are in t